# QLによるビン取りゲーム（学習）

## ライブラリのインストール

In [1]:
import numpy as np

## シミュレータクラスの設定

In [2]:
class MySimulator():
    def __init__(self, num_bottle):
        self._nb = num_bottle
        self.reset()
#初期化
    def reset(self):
        self._state = 0
        return self._state
#行動による状態変化
    def step(self, action, turn):
        self._state = self._state + action + 1
        rewards = [0,0]
        done = False
        if (self._state>=self._nb):
            self._state = self._nb
            rewards[turn] = -1
            rewards[(turn+1)%2] = 1
            done = True
        return self._state, rewards, done

## Q値クラスの設定

In [3]:
class MyQTable():
    def __init__(self, num_bottle):
        self._nb = num_bottle
        QV0=np.zeros((3,self._nb+1), dtype=np.float32)
        QV1=np.zeros((3,self._nb+1), dtype=np.float32)
        self._QVs = [QV0, QV1]
#    self._Qtable = np.zeros((2, 2))
#行動の選択
    def get_action(self, state, epsilon, turn):
        qv= self._QVs[turn]
        if epsilon > np.random.uniform(0, 1):#徐々に最適行動のみをとる、ε-greedy法
            next_action = np.random.choice([0, 1, 2])
        else:
            a = np.where(qv[:,state]==qv[:,state].max())[0]
            next_action = np.random.choice(a)
        return next_action
    def update_Qtable(self, act, reward, pos, pos_old, turn):
        qv= self._QVs[turn]
        alpha = 0.5
        gamma = 0.9
        maxQ = np.max(qv[:,pos])
        qv[act,pos_old] = (1-alpha)*qv[act,pos_old]+alpha*(reward + gamma*maxQ);
        self._QVs[turn] = qv
    def show_Qtable(self):
        print('Agent0')
        print(self._QVs[0].T)
        print('Agent1')
        print(self._QVs[1].T)
        for j in range(2):
            print("Agent", j)
            qv = self._QVs[j]
            for i in range(self._nb):
                a = np.where(qv[:,i]==qv[:,i].max())[0]
#                a = np.where(self._QVs[j][i]==self._QVs[j][i].max())[0]
                print('残り本数',self._nb-i,'取る数',a+1,'必勝法',(self._nb-i-1)%4,'なんでもよい' if (self._nb-i-1)%4 == 0 else \
                      ('不定' if a.size >1 else ('正解' if (self._nb-i-1)%4 == a+1 else '不正解')))


## その他の設定

In [4]:
num_episodes = 100  #総試行回数
num_bottle = 9
env = MySimulator(num_bottle)
tab = MyQTable(num_bottle)

## 学習

In [5]:
for episode in range(num_episodes):  #試行数分繰り返す
    state = env.reset()
    state_old = [0,0]
    rewards = [0,0]
    actions = [0,0]
    epsilon = 0.5 * (1 / (episode + 1))
    while(1):
        actions[0] = tab.get_action(state, epsilon, turn=0)
        state_old[0] = state
        state, rewards, done = env.step(actions[0], 0)
        tab.update_Qtable(actions[1], rewards[1], state, state_old[1], turn=1)
        if (done==True):
            tab.update_Qtable(actions[0], rewards[0], state, state_old[0], turn=0)
            print('{} : 0 Loose, 1 Win!!'.format(episode))
            break
        actions[1] = tab.get_action(state, epsilon, turn=1)
        state_old[1] = state
        state, rewards, done = env.step(actions[1], 1)
        tab.update_Qtable(actions[0], rewards[0], state, state_old[0], turn=0)
        if (done==True):
            tab.update_Qtable(actions[1], rewards[1], state, state_old[1], turn=1)
            print('{} : 0 Win!!, 1 Loose'.format(episode))
            break
tab.show_Qtable()

np.savetxt('Qvalue0.txt', tab._QVs[0])
np.savetxt('Qvalue1.txt', tab._QVs[1])


0 : 0 Win!!, 1 Loose
1 : 0 Win!!, 1 Loose
2 : 0 Win!!, 1 Loose
3 : 0 Loose, 1 Win!!
4 : 0 Loose, 1 Win!!
5 : 0 Loose, 1 Win!!
6 : 0 Win!!, 1 Loose
7 : 0 Win!!, 1 Loose
8 : 0 Loose, 1 Win!!
9 : 0 Loose, 1 Win!!
10 : 0 Loose, 1 Win!!
11 : 0 Loose, 1 Win!!
12 : 0 Loose, 1 Win!!
13 : 0 Loose, 1 Win!!
14 : 0 Loose, 1 Win!!
15 : 0 Win!!, 1 Loose
16 : 0 Loose, 1 Win!!
17 : 0 Loose, 1 Win!!
18 : 0 Win!!, 1 Loose
19 : 0 Win!!, 1 Loose
20 : 0 Win!!, 1 Loose
21 : 0 Win!!, 1 Loose
22 : 0 Loose, 1 Win!!
23 : 0 Win!!, 1 Loose
24 : 0 Win!!, 1 Loose
25 : 0 Loose, 1 Win!!
26 : 0 Loose, 1 Win!!
27 : 0 Loose, 1 Win!!
28 : 0 Loose, 1 Win!!
29 : 0 Loose, 1 Win!!
30 : 0 Loose, 1 Win!!
31 : 0 Loose, 1 Win!!
32 : 0 Loose, 1 Win!!
33 : 0 Loose, 1 Win!!
34 : 0 Loose, 1 Win!!
35 : 0 Loose, 1 Win!!
36 : 0 Loose, 1 Win!!
37 : 0 Loose, 1 Win!!
38 : 0 Loose, 1 Win!!
39 : 0 Loose, 1 Win!!
40 : 0 Loose, 1 Win!!
41 : 0 Loose, 1 Win!!
42 : 0 Loose, 1 Win!!
43 : 0 Loose, 1 Win!!
44 : 0 Loose, 1 Win!!
45 : 0 Loose, 1 Win!